In [370]:
def to_two_complement(n):
    if (n < 0):
        n = (-n) ^ 0xFF
        n += 1
    
    return n

def to_hex(n):
    return hex(n)[2:].upper()

def to_dec(n):
    return int(n, 16)

S = 34*256 + 255
print(to_hex(S))
print(to_dec("8"))

print(to_hex(((1) ^ 0xFF) + 1))

22FF
8
FF


In [371]:
VAR_TABLE = {
    "i": 1,
    "start": 10
}

def is_int(s):
    for i in s:
        if not i.isdigit():
            return False
    return True

def parse_var(v):
    if isinstance(v, int):
        return v
    elif v == "":
        return 0
    elif v in VAR_TABLE:
        return VAR_TABLE[v]
    elif is_int(v):
        return int(v)
    elif "-" in v:
        p = v.find("-")
        #print(p)
        return parse_var(v[0:p]) - parse_var(v[p+1:])
    elif "+" in v:
        return sum(list(map(parse_var, v.split("+"))))
    else:
        raise Exception()

In [372]:
parse_var("11-i")

10

In [373]:
class Instruction(object):
    OP2CODE = {
        "load-memory": 1,
        "load-pattern": 2,
        "store": 3,
        "add": 5,
        "jump": 11,
    }
    OP_OPERAND = {
        "load-memory": "48",
        "load-pattern": "48",
        "store": "48",
        "add": "444",
        "jump": "48",
    }
    def __init__(self, operation, operand = []):
        print ("op = %s operand = %s" % (operation, operand))
        if (len(operand) != len(self.OP_OPERAND[operation])):
            raise Exception("Incorrect operation: %s %s" % (operation, operand))

        self.operation = operation
        self.operand = list(operand)

    def process_variable(self):
        for i in range(0, len(self.operand)):
            try:
                self.operand[i] = parse_var(self.operand[i])
            except:
                print("Failing to parse %s" % self.operand[i])
                pass
            
    def to_machine_code(self):
        n = self.OP2CODE[self.operation]
        pat = self.OP_OPERAND[self.operation]

        for i in range(0, len(self.operand)):
            #print(n)
            n *= 2**to_dec(pat[i])
            # print(to_dec(pat[i]))
            # print("%s %s" % (self.operand[i], parse_var(self.operand[i])))
            n += to_two_complement(parse_var(self.operand[i]))
        
        #print(n)
        return to_hex(n)

In [374]:
ins = Instruction("jump", ["i", "start"])

op = jump operand = ['i', 'start']


In [375]:
ins.to_machine_code()

'B10A'

In [376]:
class MachineCode(object):
    def __init__(self, src):
        self.instructions = []
        for line in src.splitlines():
            ops = line.split()
            if len(ops) == 0:
                continue
            
            if ops[0] == "using":
                VAR_TABLE[ops[1]] = int(ops[-1])
            elif ops[0] == "tag":
                VAR_TABLE[ops[1]] = len(self.instructions) * 2
            else:
                ins = Instruction(ops[0], ops[1:])
                ins.process_variable()
                self.instructions.append(ins)
            
        VAR_TABLE["__code_length"] = len(self.instructions) * 2
            
    def __repr__(self):
        s = ""
        for i in self.instructions:
            s += i.to_machine_code() + "\n"
        return s

In [377]:
code = """
using i           register 1
using minus1      register 2
using tmp         register 3
using zero        register 0
using to          register 5

load-pattern minus1 -1
load-pattern i __code_length
tag start
add i i minus1
store i load_pos+1
tag load_pos
load-memory tmp 170
load-pattern to 70
add to to i
store to store_pos+1
tag store_pos
store tmp 170
jump i 70
jump zero start"""

In [378]:
VAR_TABLE = {}
mc = MachineCode(code)
print(VAR_TABLE)

op = load-pattern operand = ['minus1', '-1']
op = load-pattern operand = ['i', '__code_length']
Failing to parse __code_length
op = add operand = ['i', 'i', 'minus1']
op = store operand = ['i', 'load_pos+1']
Failing to parse load_pos+1
op = load-memory operand = ['tmp', '170']
op = load-pattern operand = ['to', '70']
op = add operand = ['to', 'to', 'i']
op = store operand = ['to', 'store_pos+1']
Failing to parse store_pos+1
op = store operand = ['tmp', '170']
op = jump operand = ['i', '70']
op = jump operand = ['zero', 'start']
{'load_pos': 8, 'start': 4, 'store_pos': 16, '__code_length': 22, 'i': 1, 'zero': 0, 'to': 5, 'tmp': 3, 'minus1': 2}


In [379]:
print(mc)

22FF
2116
5112
3109
13AA
2546
5551
3511
33AA
B146
B004

